In [1]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import matplotlib.pyplot as plt
import numpy as np
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.nn as nn
import torchvision.models as models

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Define transformations for test data
test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to model's input size
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Standard normalization
])

# Load test dataset
test_dataset = datasets.ImageFolder(root="/content/drive/MyDrive/Projects/HDA Proj/test", transform=test_transforms)
test_loader = DataLoader(test_dataset, batch_size=96, shuffle=False)

##DenseNet

In [ ]:
class OCTDenseNetModel:
    def __init__(self, num_classes=4, learning_rate=0.0001):
        # Load pretrained DenseNet121 model
        self.model = models.densenet121(pretrained=True)

        # Modify the classifier layer
        in_features = self.model.classifier.in_features
        self.model.classifier = nn.Linear(in_features, num_classes)

        # Define loss function and optimizer
        self.criterion = nn.CrossEntropyLoss()
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=learning_rate)

    def get_model(self):
        return self.model, self.criterion, self.optimizer

In [ ]:
import time
from tqdm import tqdm

# Load your model
model1 = OCTDenseNetModel(num_classes=4).get_model()[0]
model1.load_state_dict(torch.load("/content/drive/MyDrive/Projects/HDA Proj/models/DenseNet_oct_mini_model.pth"))  # Load model weights
model1.to('cuda')  # Move model to GPU if available
model1.eval()  # Set to evaluation mode

# Initialize lists to store true and predicted labels
all_labels = []
all_predictions = []

# Start timer
start_time = time.time()

# Initialize progress bar for visualization
with torch.no_grad():
    for images, labels in tqdm(test_loader, desc="Testing Progress", unit="batch"):
        # Move data to the appropriate device, e.g., GPU
        images, labels = images.to('cuda'), labels.to('cuda')

        # Forward pass
        outputs = model1(images)
        _, predicted = torch.max(outputs, 1)

        # Store labels and predictions
        all_labels.extend(labels.cpu().numpy())
        all_predictions.extend(predicted.cpu().numpy())

# Calculate total time taken
elapsed_time = time.time() - start_time
print(f"Testing completed in {elapsed_time:.2f} seconds")


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /root/.cache/torch/hub/checkpoints/densenet121-a639ec97.pth
100%|██████████| 30.8M/30.8M [00:00<00:00, 189MB/s]
<ipython-input-6-39a34771dddd>:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. 

Testing completed in 604.35 seconds


In [ ]:
# Calculate metrics
accuracy = accuracy_score(all_labels, all_predictions)
precision = precision_score(all_labels, all_predictions, average='weighted')
recall = recall_score(all_labels, all_predictions, average='weighted')
f1 = f1_score(all_labels, all_predictions, average='weighted')

print(f'Test Accuracy: {accuracy:.4f}')
print(f'Weighted Precision: {precision:.4f}')
print(f'Weighted Recall: {recall:.4f}')
print(f'Weighted F1 Score: {f1:.4f}')

Test Accuracy: 0.9900
Weighted Precision: 0.9902
Weighted Recall: 0.9900
Weighted F1 Score: 0.9900


##ViT

In [ ]:
!pip install transformers

In [ ]:
from transformers import ViTForImageClassification
import torch
import time
from tqdm import tqdm

# Load your model
model2 = ViTForImageClassification.from_pretrained(
    "google/vit-base-patch16-224-in21k", num_labels=4
)
model2.load_state_dict(torch.load("/content/drive/MyDrive/Projects/HDA Proj/models/vit_oct_mini_model.pth", weights_only=True))  # Load model weights
model2.to('cuda')  # Move model to GPU if available
model2.eval()  # Set to evaluation mode

# Initialize lists to store true and predicted labels
all_labels = []
all_predictions = []

# Start timer
start_time = time.time()

# Initialize progress bar for visualization
with torch.no_grad():
    for images, labels in tqdm(test_loader, desc="Testing Progress", unit="batch"):
        # Move data to the appropriate device, e.g., GPU
        images, labels = images.to('cuda'), labels.to('cuda')

        # Forward pass
        outputs = model2(images)
        logits = outputs.logits  # Extract logits
        _, predicted = torch.max(logits, 1)  # Compute predictions

        # Store labels and predictions
        all_labels.extend(labels.cpu().numpy())
        all_predictions.extend(predicted.cpu().numpy())

# Calculate total time taken
elapsed_time = time.time() - start_time
print(f"Testing completed in {elapsed_time:.2f} seconds")


Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Testing Progress: 100%|██████████| 11/11 [00:20<00:00,  1.82s/batch]

Testing completed in 20.04 seconds


In [ ]:
# Calculate metrics
accuracy = accuracy_score(all_labels, all_predictions)
precision = precision_score(all_labels, all_predictions, average='weighted')
recall = recall_score(all_labels, all_predictions, average='weighted')
f1 = f1_score(all_labels, all_predictions, average='weighted')

print(f'Test Accuracy: {accuracy:.4f}')
print(f'Weighted Precision: {precision:.4f}')
print(f'Weighted Recall: {recall:.4f}')
print(f'Weighted F1 Score: {f1:.4f}')

Test Accuracy: 0.9890
Weighted Precision: 0.9894
Weighted Recall: 0.9890
Weighted F1 Score: 0.9890


##EfficientNet

In [ ]:
from transformers import ViTForImageClassification
# Load your model
model3 = models.efficientnet_b0(pretrained=False)
model3.classifier[1] = nn.Linear(model3.classifier[1].in_features, 4)
model3.load_state_dict(torch.load("/content/drive/MyDrive/Projects/HDA Proj/models/efficientnet_oct_model.pth"))  # Load model weights
model3.to('cuda')  # Move model to GPU if available
model3.eval()  # Set to evaluation mode

# Initialize lists to store true and predicted labels
all_labels = []
all_predictions = []

# Start timer
start_time = time.time()

# Initialize progress bar for visualization
with torch.no_grad():
    for images, labels in tqdm(test_loader, desc="Testing Progress", unit="batch"):
        # Move data to the appropriate device, e.g., GPU
        images, labels = images.to('cuda'), labels.to('cuda')

        # Forward pass
        outputs = model3(images)
        _, predicted = torch.max(outputs, 1)

        # Store labels and predictions
        all_labels.extend(labels.cpu().numpy())
        all_predictions.extend(predicted.cpu().numpy())

# Calculate total time taken
elapsed_time = time.time() - start_time
print(f"Testing completed in {elapsed_time:.2f} seconds")



/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
<ipython-input-13-e2587870ad45>:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions

Testing completed in 9.31 seconds


In [ ]:
# Calculate metrics
accuracy = accuracy_score(all_labels, all_predictions)
precision = precision_score(all_labels, all_predictions, average='weighted')
recall = recall_score(all_labels, all_predictions, average='weighted')
f1 = f1_score(all_labels, all_predictions, average='weighted')

print(f'Test Accuracy: {accuracy:.4f}')
print(f'Weighted Precision: {precision:.4f}')
print(f'Weighted Recall: {recall:.4f}')
print(f'Weighted F1 Score: {f1:.4f}')

Test Accuracy: 0.9920
Weighted Precision: 0.9922
Weighted Recall: 0.9920
Weighted F1 Score: 0.9920


##Swin Transformer

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from transformers import SwinForImageClassification
from torch.utils.data import DataLoader
from tqdm import tqdm
import time

model4 = SwinForImageClassification.from_pretrained('microsoft/swin-tiny-patch4-window7-224')
model4.classifier = nn.Linear(model4.classifier.in_features, 4)
model4.load_state_dict(torch.load("/content/drive/MyDrive/Projects/HDA Proj/models/swin_transformer_mini_model.pth", weights_only=True))  # Load model weights
model4.to('cuda')
model4.eval()  # Set to evaluation mode

# Initialize lists to store true and predicted labels
all_labels = []
all_predictions = []

# Start timer
start_time = time.time()

# Initialize progress bar for visualization
with torch.no_grad():
    for images, labels in tqdm(test_loader, desc="Testing Progress", unit="batch"):
        # Move data to the appropriate device, e.g., GPU
        images, labels = images.to('cuda'), labels.to('cuda')

        # Forward pass
        outputs = model4(images)
        logits = outputs.logits  # Extract logits
        _, predicted = torch.max(logits, 1)  # Compute predictions

        # Store labels and predictions
        all_labels.extend(labels.cpu().numpy())
        all_predictions.extend(predicted.cpu().numpy())

# Calculate total time taken
elapsed_time = time.time() - start_time
print(f"Testing completed in {elapsed_time:.2f} seconds")


Testing Progress: 100%|██████████| 11/11 [14:24<00:00, 78.59s/batch]

Testing completed in 864.50 seconds


In [6]:
# Calculate metrics
accuracy = accuracy_score(all_labels, all_predictions)
precision = precision_score(all_labels, all_predictions, average='weighted')
recall = recall_score(all_labels, all_predictions, average='weighted')
f1 = f1_score(all_labels, all_predictions, average='weighted')

print(f'Test Accuracy: {accuracy:.4f}')
print(f'Weighted Precision: {precision:.4f}')
print(f'Weighted Recall: {recall:.4f}')
print(f'Weighted F1 Score: {f1:.4f}')

Test Accuracy: 0.9820
Weighted Precision: 0.9828
Weighted Recall: 0.9820
Weighted F1 Score: 0.9820
